In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import matplotlib
import time
import numpy as np
%matplotlib inline

import pandas as pd
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 100



In [2]:
#Model File Directory
WORK_DIR = '/var/spark/ml_files/'

#Label Keys
LABEL = "success"

MODEL_TYPE = "ML-BR"

start_date = '2018-01-01'
# start_date = '2019-01-01'

end_date = '2019-07-31'
# end_date = '2019-06-30'

# desc = '%s_%s_for_calendar_retry_attempt'.format(start_date, end_date)

In [3]:
# from cassandra
from cassandra.cluster import Cluster
cassandra_endpoint = '10.81.12.121' #'10.62.1.118'
# cassandra_endpoint = '10.224.12.32'
cluster = Cluster([cassandra_endpoint])
# session = cluster.connect('subs')


In [4]:
FEATURES_CANDIDATES = ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 
                 'day_of_month', 'site_id', 'failed_decline_type', 'merchant_number', 
                'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num', 'failed_day_of_month', 
                'payment_currency', 'days_between', 'failed_response_code', 'payment_amount_usd', 'date_increment', 'transaction_hour']

USECOLS = FEATURES_CANDIDATES +  ['new_status','subscription_id', 'success', 'cid' ,'bank_name','added_expiry_years', 'failed_response_message','date_increment', 'received_date', 'billing_country', 'transaction_date_in_string', 'cc_expiration_date', 'failed_attempt_date']

In [5]:
retry_success_2018 = pd.read_csv(WORK_DIR + 'calendar_retry_01_to_12.csv', usecols=USECOLS)
# retry_success_2018 = retry_success_2018[retry_success_2018['success'] == 1]
retry_success_2018 = retry_success_2018[~(retry_success_2018['received_date'] >= '2018-08-01')]
retry_success_2018 = retry_success_2018[~(retry_success_2018['new_status'] == 'Reversed')]
retry_success_2018.shape

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(4002072, 33)

In [5]:
# all_files = ['calendar_retry_01_to_12.csv', 'calendar_retry_2019_01_to_03.csv', 'calendar_retry_2019_04.csv', 'calendar_retry_2019_05.csv', 'calendar_retry_2019_06.csv', 'calendar_retry_2019_07.csv']
# all_files = ['calendar_retry_2018_12_new.csv', 'calendar_retry_2019_01_to_04_new.csv', 'calendar_retry_2019_05_06_new.csv', 'calendar_retry_2019_07_new.csv']
all_files = ['calendar_retry_2019_01_to_04_new.csv', 'calendar_retry_2019_05_06_new.csv']

retry_success_2019 =  pd.concat((pd.read_csv(WORK_DIR + file, usecols=USECOLS) for file in all_files) , ignore_index=True)
retry_success_2019 = retry_success_2019[~(retry_success_2019['new_status'] == 'Reversed')]
retry_success_2019.shape

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DtypeWarning: Columns (6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  """


(3855915, 33)

In [6]:
retry_success_2019_06 = pd.read_csv(WORK_DIR + 'calendar_retry_2019_06.csv', usecols=USECOLS)
retry_success_2019_06 = retry_success_2019_06[~(retry_success_2019_06['new_status'] == 'Reversed')]
retry_success_2019_06.shape

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(517648, 33)

In [7]:
retry_success_2019_07 = pd.read_csv(WORK_DIR + 'calendar_retry_2019_07_new.csv', usecols=USECOLS)

retry_success_2019_07.shape

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(629027, 33)

In [8]:
retry_success_2019_08_to_09 = pd.read_csv(WORK_DIR + 'calendar_retry_2019_08_to_09_new.csv', usecols=USECOLS)
retry_success_2019_08_to_09 = retry_success_2019_08_to_09[~(retry_success_2019_08_to_09['new_status'] == 'Reversed')]
retry_success_2019_08_to_09 = retry_success_2019_08_to_09[~retry_success_2019_08_to_09['payment_amount_usd'].isna()]
retry_success_08 = retry_success_2019_08_to_09[retry_success_2019_08_to_09['received_date'].str.startswith('2019-08')]
retry_success_09 = retry_success_2019_08_to_09[retry_success_2019_08_to_09['received_date'].str.startswith('2019-09')]
retry_success_09.head()

,received_date,site_id,subscription_id,added_expiry_years,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,funding_source,issuer_country,merchant_number,new_status,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,transaction_date_in_string,transaction_hour,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between
3,2019-09-04,adbehbr,10030264201,NaN,BANCO SANTANDER (BRASIL) S.A.,BR,542820.0,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,Credit - Credit Hybrid (meaning it has pin capability also),0623,NaN,NaN,4,Credit,BR,75FA81DE-5AC5-5DCF-E97C-8F80A08ADE1E,Declined,8.379192,BRL,MasterCard,pagador,2,2019-09-04 02:15:44,2,0,2019-08-28 03:15:56,28,530.0,7: Do Not Honor,do_not_honor,7
4,2019-09-04,adbehap,10031043801,NaN,Bank of Ayudhya Public Company Limited,TH,409344.0,Visa,Visa Signature,Consumer,Credit - True credit (No PIN/Signature capability),0222,NaN,NaN,4,Credit,TH,DR_IRELAND_ROW,Declined,11.605974,THB,Visa,adyen,2,2019-09-04 01:53:19,1,0,2019-08-28 02:54:13,28,530.0,27001: Transaction refused,txn_refused,7
9,2019-09-05,adbehap,10062414901,NaN,KASIKORNBANK PUBLIC COMPANY LIMITED,TH,540488.0,MasterCard,Gold MasterCard Card,Consumer,Credit - True credit (No PIN/Signature capability),0420,NaN,NaN,5,Credit,TH,DR_IRELAND_ROW,Completed,11.628151,THB,MasterCard,adyen,5,2019-09-05 01:27:11,1,1,2019-08-29 01:08:18,29,530.0,27001: Transaction refused,txn_refused,7
14,2019-09-04,adbehbr,10148099001,3,BANCO BRADESCO S.A.,BR,536805.0,MasterCard,Platinum MasterCard Card,Consumer,Credit - True credit (No PIN/Signature capability),0822,NaN,NaN,4,Credit,BR,75FA81DE-5AC5-5DCF-E97C-8F80A08ADE1E,Declined,25.137576,BRL,MasterCard,pagador,2,2019-09-04 02:15:45,2,0,2019-08-28 02:46:02,28,530.0,7: Do Not Honor,do_not_honor,7
32,2019-09-05,avast,10308029501,3,NaN,JP,354008.0,Discover,Consumer Credit - Premium,Consumer,Credit - True credit (No PIN/Signature capability),0722,NaN,NaN,5,Credit,JP,DR_IRELAND_ROW,Declined,42.177766,JPY,JCB,adyen,2,2019-09-05 00:47:23,0,0,2019-08-26 00:50:26,26,530.0,27001: Transaction refused,txn_refused,10


In [9]:
# retry_success = pd.concat([retry_success, retry_success_2018], ignore_index=True)
retry_success_2019 = pd.concat([retry_success_2019, retry_success_2019_07], ignore_index=True)

retry_success_2019.shape

(4484942, 33)

In [10]:
retry_success = pd.concat([retry_success_2019, retry_success_2018], ignore_index=True)

retry_success.shape

NameError: name 'retry_success_2018' is not defined

In [10]:
FEATURES_FLOAT = ['bin', 'renew_att_num', 'failed_response_code']
# FEATURES_FLOAT = ['date_increment']

def handle_feat_float(df):
        for feat in FEATURES_FLOAT:
            if feat in FEATURES_FLOAT:
                df[feat] = df[feat].apply(str).str.replace('.0', '', regex=False)
        return df

# handle_feat_float(retry_success)    
handle_feat_float(retry_success_2019)    

# retry_success.head()

,received_date,site_id,subscription_id,added_expiry_years,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,funding_source,issuer_country,merchant_number,new_status,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,transaction_date_in_string,transaction_hour,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between
0,2019-04-13,kasperus,10000899501,NaN,Navy Federal Credit Union,US,400022,Visa,Visa Classic,Consumer,Debit - Debit Hybrid (PIN and Signature),0320,NaN,NaN,13,Debit,US,311009012882,Declined,119.99,USD,Visa,firstdata,2,2019-04-13 07:45:58,7,0,2019-04-08 14:15:06,8,201,14: INV ACCT NUM,Base,5
1,2019-04-18,kasperus,10000899501,NaN,Navy Federal Credit Union,US,400022,Visa,Visa Classic,Consumer,Debit - Debit Hybrid (PIN and Signature),0320,NaN,NaN,18,Debit,US,311009012882,Declined,119.99,USD,Visa,firstdata,3,2019-04-18 05:16:43,5,0,2019-04-13 07:45:58,13,201,14: INV ACCT NUM,Base,5
2,2019-04-25,kasperus,10000899501,NaN,Navy Federal Credit Union,US,400022,Visa,Visa Classic,Consumer,Debit - Debit Hybrid (PIN and Signature),0320,NaN,NaN,25,Debit,US,311009012882,Declined,119.99,USD,Visa,firstdata,4,2019-04-25 09:15:53,9,0,2019-04-18 05:16:43,18,201,14: INV ACCT NUM,Base,7
3,2019-04-13,kasper,10001135201,0,Swisscard AECS GmbH,CH,402730,Visa,Visa Classic,Consumer,Credit - True credit (No PIN/Signature capability),1117,NaN,0.0,13,Credit,CH,1411163460-CHF-pacific,Declined,72.00,CHF,Visa,netgiro-bms,2,2019-04-13 01:25:03,1,0,2019-04-08 00:34:31,8,530,27001: Transaction refused,txn_refused,5
4,2019-04-18,kasper,10001135201,2,Swisscard AECS GmbH,CH,402730,Visa,Visa Classic,Consumer,Credit - True credit (No PIN/Signature capability),1119,NaN,3.0,18,Credit,CH,1411163460-CHF-pacific,Declined,71.40,CHF,Visa,netgiro-bms,3,2019-04-18 00:33:22,0,0,2019-04-13 01:25:05,13,530,27001: Transaction refused,txn_refused,5
5,2019-04-13,avast,10001255101,NaN,NaN,US,371558,American Express,"No Data Found', value: 'N/A",No Data found,Credit - True credit (No PIN/Signature capability),0120,NaN,NaN,13,No Data found,NaN,941000108071_00000001,Declined,49.99,USD,AmericanExpress,mes,2,2019-04-13 07:42:45,7,0,2019-04-03 15:45:20,3,530,51: DECLINED,declined,10
6,2019-04-18,avast,10001255101,NaN,NaN,US,371558,American Express,"No Data Found', value: 'N/A",No Data found,Credit - True credit (No PIN/Signature capability),0120,NaN,NaN,18,No Data found,NaN,941000108071_00000001,Declined,49.99,USD,AmericanExpress,mes,3,2019-04-18 16:45:56,16,0,2019-04-13 07:42:45,13,530,51: DECLINED,declined,5
7,2019-04-19,tmapac,10001514301,NaN,COMMONWEALTH BANK OF AUSTRALIA,AU,521729,MasterCard,Standard Debit MasterCard,Consumer,Debit - NON USA Consumer Debit - Pin Capable,1019,NaN,NaN,19,Debit,AU,DR_IRELAND_ROW,Declined,31.60,MYR,MasterCard,adyen,2,2019-04-19 12:00:11,12,0,2019-04-14 12:42:06,14,401,Blocked Card: Blocked Card,blocked_first_used,5
8,2019-04-26,tmapac,10001514301,NaN,COMMONWEALTH BANK OF AUSTRALIA,AU,521729,MasterCard,Standard Debit MasterCard,Consumer,Debit - NON USA Consumer Debit - Pin Capable,1019,NaN,NaN,26,Debit,AU,DR_IRELAND_ROW,Declined,31.60,MYR,MasterCard,adyen,3,2019-04-26 00:35:23,0,0,2019-04-19 12:00:11,19,401,Blocked Card: Blocked Card,blocked_first_used,7
9,2019-04-13,avast,10001569501,NaN,JACK HENRY & ASSOCIATES,US,551765,MasterCard,Debit MasterCard (Enhanced),Consumer,Debit - Debit Hybrid (PIN and Signature),1120,NaN,NaN,13,Debit,US,311009012882,Declined,49.99,USD,MasterCard,firstdata,2,2019-04-13 08:15:57,8,0,2019-04-03 13:15:33,3,530,349: Do Not Honor,do_not_honor,10


In [11]:
#import for training
import numpy as np
from sklearn import cross_validation
from sklearn import metrics
from sklearn.model_selection import cross_val_score

from sklearn import linear_model
from sklearn import tree
from sklearn import cross_validation
from sklearn import ensemble
from sklearn import linear_model
from sklearn import svm
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV
# from spark_sklearn import GridSearchCV
from sklearn.preprocessing import label_binarize
from sklearn.neighbors import KNeighborsClassifier

# from src.web.utils import PreProcessing
from src.web.preprocessing import PreProcessing
from src.web.encoder import EnhancedLeaveOneOutEncoder
from src.web.train_util import *

/opt/anaconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [12]:
INPUT_FEATURES = {
            "billing_country": {
                "type": "string"
            },
            "bin": {
                "type": "string"
            },
            "card_brand": {
                "type": "string"
            },
            "card_category": {
                "type": "string"
            },
            "card_class": {
                "type": "string"
            },
            "card_usage": {
                "type": "string"
            },
            "cc_expiration_date": {
                "type": "string"
            },
            "day_of_month": {
                "type": "integer"
            },
            "failed_attempt_date": {
                "type": "string"
            },
            "failed_response_code": {
                "type": "string"
            },
            "failed_response_message": {
                "type": "string"
            },
            "funding_source": {
                "type": "string"
            },
            "issuer_country": {
                "type": "string"
            },
            "merchant_number": {
                "type": "string"
            },
            "payment_amount_usd": {
                "type": "number"
            },
            "payment_currency": {
                "type": "string"
            },
            "payment_method_id": {
                "type": "string"
            },
            "payment_service_id": {
                "type": "string"
            },
            "renew_att_num": {
                "type": "integer"
            },
            "site_id": {
                "type": "string"
            },
            "transaction_date_in_string": {
                "type": "string"
            },
            "date_increment": {
                "type": "string"
            }
        }


In [13]:
bin_profile_per_date = pd.read_csv(WORK_DIR + 'bin_profile_per_date_201801_to_201907.csv')
bin_profile_per_date['bin'] = bin_profile_per_date['bin'].apply(str).str.replace('.0', '', regex=False)
max_per_date_dict = bin_profile_per_date.set_index(['bin', 'day_of_month'])['Max_99'].T.to_dict()

In [14]:
date_mid_bin = pd.read_csv(WORK_DIR + 'date_mid_bin_2019_01_to_2019_07.csv')
date_mid_bin['bin'] = date_mid_bin['bin'].apply(str).str.replace('.0', '', regex=False)
date_mid_bin_dict = date_mid_bin[date_mid_bin['nunique'] > 30].set_index(['bin', 'merchant_number','day_of_month'])['success_rate'].T.to_dict()

date_processor_bin = pd.read_csv(WORK_DIR + 'date_procesoor_bin_2019_01_to_2019_07.csv')
date_processor_bin['bin'] = date_processor_bin['bin'].apply(str).str.replace('.0', '', regex=False)
date_processor_bin_dict = date_processor_bin[date_processor_bin['nunique'] > 30].set_index(['bin', 'payment_service_id','day_of_month'])['success_rate'].T.to_dict()


date_bin = pd.read_csv(WORK_DIR + 'date_bin_2019_01_to_2019_07.csv')
date_bin['bin'] = date_bin['bin'].apply(str).str.replace('.0', '', regex=False)
date_bin_dict = date_bin[date_bin['nunique'] > 30].set_index(['bin', 'day_of_month'])['success_rate'].T.to_dict()

# bin_profile_per_date = pd.read_csv(WORK_DIR + 'bin_profile_per_date_201801_to_201907.csv')
# bin_profile_per_date['bin'] = bin_profile_per_date['bin'].apply(str).str.replace('.0', '', regex=False)
# max_per_date_dict = bin_profile_per_date.set_index(['bin', 'day_of_month'])['Max_99'].T.to_dict()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
!ls /var/spark/ml_files/

In [20]:
# days_between_decline_type = pd.read_csv(WORK_DIR + 'days_between_decline_type_2019_01_to_2019_07.csv')
# days_between_decline_type_dict = days_between_decline_type[days_between_decline_type['count'] > 15].set_index(['failed_decline_type', 'days_between'])['success_rate'].T.to_dict()

# days_between_bin_decline_type = pd.read_csv(WORK_DIR + 'days_between_bin_decline_type_2019_01_to_2019_07.csv')
# days_between_bin_decline_type['bin'] = days_between_bin_decline_type['bin'].apply(str).str.replace('.0', '', regex=False)
# days_between_bin_decline_type_dict = days_between_bin_decline_type[days_between_bin_decline_type['count'] > 15].set_index(['failed_decline_type', 'bin','days_between'])['success_rate'].T.to_dict()

# days_between_country_decline_type = pd.read_csv(WORK_DIR + 'days_between_country_decline_type_2019_01_to_2019_07.csv')
# days_between_country_decline_type_dict = days_between_country_decline_type[days_between_country_decline_type['count'] > 25].set_index(['failed_decline_type', 'issuer_country','days_between'])['success_rate'].T.to_dict()

# days_between_date_decline_type = pd.read_csv(WORK_DIR + 'days_between_date_decline_type_2019_01_to_2019_07.csv')
# days_between_date_decline_type_dict = days_between_date_decline_type[days_between_date_decline_type['count'] > 25].set_index(['failed_decline_type', 'day_of_month','days_between'])['success_rate'].T.to_dict()

# days_between_date_bin_decline_type = pd.read_csv(WORK_DIR + 'days_between_date_bin_decline_type_2019_01_to_2019_07.csv')
# days_between_date_bin_decline_type['bin'] = days_between_date_bin_decline_type['bin'].apply(str).str.replace('.0', '', regex=False)
# days_between_date_bin_decline_type_dict = days_between_date_bin_decline_type[days_between_date_bin_decline_type['count'] > 25].set_index(['failed_decline_type', 'bin', 'day_of_month','days_between'])['success_rate'].T.to_dict()

# days_between_bin_decline_code = pd.read_csv(WORK_DIR + 'days_between_bin_decline_code_2019_01_to_2019_07.csv')
# days_between_bin_decline_code['bin'] = days_between_bin_decline_code['bin'].apply(str).str.replace('.0', '', regex=False)
# days_between_bin_decline_code['failed_response_code'] = days_between_bin_decline_code['failed_response_code'].apply(str).str.replace('.0', '', regex=False)
# days_between_bin_decline_code_dict = days_between_bin_decline_code[days_between_bin_decline_code['count'] > 25].set_index(['failed_response_code', 'bin','days_between'])['success_rate'].T.to_dict()

# days_between_bin_failed_message = pd.read_csv(WORK_DIR + 'days_between_bin_failed_message_2019_01_to_2019_07.csv')
# days_between_bin_failed_message['bin'] = days_between_bin_failed_message['bin'].apply(str).str.replace('.0', '', regex=False)
# days_between_bin_failed_message_dict = days_between_bin_failed_message[days_between_bin_failed_message['count'] > 25].set_index(['failed_response_message', 'bin','days_between'])['success_rate'].T.to_dict()

# days_between_site_decline_type = pd.read_csv(WORK_DIR + 'days_between_site_decline_type_2019_01_to_2019_07.csv')
# days_between_site_decline_type_dict = days_between_site_decline_type[days_between_site_decline_type['count'] > 25].set_index(['failed_decline_type', 'site_id','days_between'])['success_rate'].T.to_dict()


# days_between_site_decline_code = pd.read_csv(WORK_DIR + 'days_between_site_decline_code_2019_01_to_2019_07.csv')
# days_between_site_decline_code['failed_response_code'] = days_between_site_decline_code['failed_response_code'].apply(str).str.replace('.0', '', regex=False)
# days_between_site_decline_code_dict = days_between_site_decline_code[days_between_site_decline_code['count'] > 25].set_index(['failed_response_code', 'site_id','days_between'])['success_rate'].T.to_dict()

# days_between_att_num_decline_type = pd.read_csv(WORK_DIR + 'days_between_att_num_decline_type_2019_01_to_2019_07.csv')
# days_between_att_num_decline_type['renew_att_num'] = days_between_att_num_decline_type['renew_att_num'].apply(str).str.replace('.0', '', regex=False)
# days_between_att_num_decline_type_dict = days_between_att_num_decline_type[days_between_att_num_decline_type['count'] > 25].set_index(['failed_decline_type', 'renew_att_num','days_between'])['success_rate'].T.to_dict()

# days_between_country_site = pd.read_csv(WORK_DIR + 'days_between_country_site_2019_01_to_2019_07.csv')
# days_between_country_site_dict = days_between_country_site[days_between_country_site['nunique'] > 25].set_index(['site_id', 'issuer_country','days_between'])['success_rate'].T.to_dict()


In [15]:
days_between_decline_type = pd.read_csv(WORK_DIR + 'days_between_decline_type_2018_01_to_2019_07.csv')
days_between_decline_type_dict = days_between_decline_type[days_between_decline_type['count'] > 15].set_index(['failed_decline_type', 'days_between'])['success_rate'].T.to_dict()

days_between_bin_decline_type = pd.read_csv(WORK_DIR + 'days_between_bin_decline_type_2018_01_to_2019_07.csv')
days_between_bin_decline_type['bin'] = days_between_bin_decline_type['bin'].apply(str).str.replace('.0', '', regex=False)
days_between_bin_decline_type_dict = days_between_bin_decline_type[days_between_bin_decline_type['count'] > 15].set_index(['failed_decline_type', 'bin','days_between'])['success_rate'].T.to_dict()

days_between_country_decline_type = pd.read_csv(WORK_DIR + 'days_between_country_decline_type_2018_01_to_2019_07.csv')
days_between_country_decline_type_dict = days_between_country_decline_type[days_between_country_decline_type['count'] > 25].set_index(['failed_decline_type', 'issuer_country','days_between'])['success_rate'].T.to_dict()

days_between_date_decline_type = pd.read_csv(WORK_DIR + 'days_between_date_decline_type_2018_01_to_2019_07.csv')
days_between_date_decline_type_dict = days_between_date_decline_type[days_between_date_decline_type['count'] > 25].set_index(['failed_decline_type', 'day_of_month','days_between'])['success_rate'].T.to_dict()

days_between_date_bin_decline_type = pd.read_csv(WORK_DIR + 'days_between_date_bin_decline_type_2018_01_to_2019_07.csv')
days_between_date_bin_decline_type['bin'] = days_between_date_bin_decline_type['bin'].apply(str).str.replace('.0', '', regex=False)
days_between_date_bin_decline_type_dict = days_between_date_bin_decline_type[days_between_date_bin_decline_type['count'] > 25].set_index(['failed_decline_type', 'bin', 'day_of_month','days_between'])['success_rate'].T.to_dict()

days_between_bin_decline_code = pd.read_csv(WORK_DIR + 'days_between_bin_decline_code_2018_01_to_2019_07.csv')
days_between_bin_decline_code['bin'] = days_between_bin_decline_code['bin'].apply(str).str.replace('.0', '', regex=False)
days_between_bin_decline_code['failed_response_code'] = days_between_bin_decline_code['failed_response_code'].apply(str).str.replace('.0', '', regex=False)
days_between_bin_decline_code_dict = days_between_bin_decline_code[days_between_bin_decline_code['count'] > 25].set_index(['failed_response_code', 'bin','days_between'])['success_rate'].T.to_dict()

days_between_bin_failed_message = pd.read_csv(WORK_DIR + 'days_between_bin_failed_message_2018_01_to_2019_07.csv')
days_between_bin_failed_message['bin'] = days_between_bin_failed_message['bin'].apply(str).str.replace('.0', '', regex=False)
days_between_bin_failed_message_dict = days_between_bin_failed_message[days_between_bin_failed_message['count'] > 25].set_index(['failed_response_message', 'bin','days_between'])['success_rate'].T.to_dict()

days_between_site_decline_type = pd.read_csv(WORK_DIR + 'days_between_site_decline_type_2018_01_to_2019_07.csv')
days_between_site_decline_type_dict = days_between_site_decline_type[days_between_site_decline_type['count'] > 25].set_index(['failed_decline_type', 'site_id','days_between'])['success_rate'].T.to_dict()


days_between_site_decline_code = pd.read_csv(WORK_DIR + 'days_between_site_decline_code_2018_01_to_2019_07.csv')
days_between_site_decline_code['failed_response_code'] = days_between_site_decline_code['failed_response_code'].apply(str).str.replace('.0', '', regex=False)
days_between_site_decline_code_dict = days_between_site_decline_code[days_between_site_decline_code['count'] > 25].set_index(['failed_response_code', 'site_id','days_between'])['success_rate'].T.to_dict()

days_between_att_num_decline_type = pd.read_csv(WORK_DIR + 'days_between_att_num_decline_type_2018_01_to_2019_07.csv')
days_between_att_num_decline_type['renew_att_num'] = days_between_att_num_decline_type['renew_att_num'].apply(str).str.replace('.0', '', regex=False)
days_between_att_num_decline_type_dict = days_between_att_num_decline_type[days_between_att_num_decline_type['count'] > 25].set_index(['failed_decline_type', 'renew_att_num','days_between'])['success_rate'].T.to_dict()

days_between_country_site = pd.read_csv(WORK_DIR + 'days_between_country_site_2019_01_to_2019_07.csv')
days_between_country_site_dict = days_between_country_site[days_between_country_site['nunique'] > 25].set_index(['site_id', 'issuer_country','days_between'])['success_rate'].T.to_dict()


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
date_country_site = pd.read_csv(WORK_DIR + 'date_country_site_2019_01_to_2019_07.csv')
date_country_site_dict = date_country_site[date_country_site['nunique'] > 25].set_index(['site_id', 'issuer_country','day_of_month'])['success_rate'].T.to_dict()


In [20]:
retry_success

,received_date,site_id,subscription_id,added_expiry_years,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,funding_source,issuer_country,merchant_number,new_status,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,transaction_date_in_string,transaction_hour,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between
0,2019-04-13,kasperus,10000899501,NaN,Navy Federal Credit Union,US,400022,Visa,Visa Classic,Consumer,Debit - Debit Hybrid (PIN and Signature),0320,NaN,NaN,13,Debit,US,311009012882,Declined,119.990000,USD,Visa,firstdata,2,2019-04-13 07:45:58,7,0,2019-04-08 14:15:06,8,201,14: INV ACCT NUM,Base,5
1,2019-04-18,kasperus,10000899501,NaN,Navy Federal Credit Union,US,400022,Visa,Visa Classic,Consumer,Debit - Debit Hybrid (PIN and Signature),0320,NaN,NaN,18,Debit,US,311009012882,Declined,119.990000,USD,Visa,firstdata,3,2019-04-18 05:16:43,5,0,2019-04-13 07:45:58,13,201,14: INV ACCT NUM,Base,5
2,2019-04-25,kasperus,10000899501,NaN,Navy Federal Credit Union,US,400022,Visa,Visa Classic,Consumer,Debit - Debit Hybrid (PIN and Signature),0320,NaN,NaN,25,Debit,US,311009012882,Declined,119.990000,USD,Visa,firstdata,4,2019-04-25 09:15:53,9,0,2019-04-18 05:16:43,18,201,14: INV ACCT NUM,Base,7
3,2019-04-13,kasper,10001135201,0,Swisscard AECS GmbH,CH,402730,Visa,Visa Classic,Consumer,Credit - True credit (No PIN/Signature capability),1117,NaN,0.0,13,Credit,CH,1411163460-CHF-pacific,Declined,72.000000,CHF,Visa,netgiro-bms,2,2019-04-13 01:25:03,1,0,2019-04-08 00:34:31,8,530,27001: Transaction refused,txn_refused,5
4,2019-04-18,kasper,10001135201,2,Swisscard AECS GmbH,CH,402730,Visa,Visa Classic,Consumer,Credit - True credit (No PIN/Signature capability),1119,NaN,3.0,18,Credit,CH,1411163460-CHF-pacific,Declined,71.400000,CHF,Visa,netgiro-bms,3,2019-04-18 00:33:22,0,0,2019-04-13 01:25:05,13,530,27001: Transaction refused,txn_refused,5
5,2019-04-13,avast,10001255101,NaN,NaN,US,371558,American Express,"No Data Found', value: 'N/A",No Data found,Credit - True credit (No PIN/Signature capability),0120,NaN,NaN,13,No Data found,NaN,941000108071_00000001,Declined,49.990000,USD,AmericanExpress,mes,2,2019-04-13 07:42:45,7,0,2019-04-03 15:45:20,3,530,51: DECLINED,declined,10
6,2019-04-18,avast,10001255101,NaN,NaN,US,371558,American Express,"No Data Found', value: 'N/A",No Data found,Credit - True credit (No PIN/Signature capability),0120,NaN,NaN,18,No Data found,NaN,941000108071_00000001,Declined,49.990000,USD,AmericanExpress,mes,3,2019-04-18 16:45:56,16,0,2019-04-13 07:42:45,13,530,51: DECLINED,declined,5
7,2019-04-19,tmapac,10001514301,NaN,COMMONWEALTH BANK OF AUSTRALIA,AU,521729,MasterCard,Standard Debit MasterCard,Consumer,Debit - NON USA Consumer Debit - Pin Capable,1019,NaN,NaN,19,Debit,AU,DR_IRELAND_ROW,Declined,31.600000,MYR,MasterCard,adyen,2,2019-04-19 12:00:11,12,0,2019-04-14 12:42:06,14,401,Blocked Card: Blocked Card,blocked_first_used,5
8,2019-04-26,tmapac,10001514301,NaN,COMMONWEALTH BANK OF AUSTRALIA,AU,521729,MasterCard,Standard Debit MasterCard,Consumer,Debit - NON USA Consumer Debit - Pin Capable,1019,NaN,NaN,26,Debit,AU,DR_IRELAND_ROW,Declined,31.600000,MYR,MasterCard,adyen,3,2019-04-26 00:35:23,0,0,2019-04-19 12:00:11,19,401,Blocked Card: Blocked Card,blocked_first_used,7
9,2019-04-13,avast,10001569501,NaN,JACK HENRY & ASSOCIATES,US,551765,MasterCard,Debit MasterCard (Enhanced),Consumer,Debit - Debit Hybrid (PIN and Signature),1120,NaN,NaN,13,Debit,US,311009012882,Declined,49.990000,USD,MasterCard,firstdata,2,2019-04-13 08:15:57,8,0,2019-04-03 13:15:33,3,530,349: Do Not Honor,do_not_honor,10


In [17]:
original_size = len(retry_success_2019)
balanced_size = len(retry_success_2019)
fail_size = retry_success_2019[LABEL].value_counts(normalize=True)[0.0]
success_size =  retry_success_2019[LABEL].value_counts(normalize=True)[1.0]

# original_size = len(retry_success)
# balanced_size = len(retry_success)
# fail_size = retry_success[LABEL].value_counts(normalize=True)[0.0]
# success_size =  retry_success[LABEL].value_counts(normalize=True)[1.0]

In [39]:
# FEATURES_CAT = ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 
#                  'day_of_month', 'site_id', 'billing_country', 'failed_decline_type', 'merchant_number', 
#                 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num', 'failed_day_of_month', 
#                 'payment_currency', 'days_between', 'failed_response_code', 'bank_code', 'expiration_date_changed', 'processor_mid_changed', 'failed_payment_service_id', 'failed_merchant_number', 'failed_cid']


FEATURES_CAT = [ 'merchant_number', 'day_of_month' ] 

FEATURES_FLOAT = ['bin', 'renew_att_num', 'failed_response_code', 'date_increment']
FEATURES_NUM = []
FEATURES_NUM_CALCULATED = []  


FEATURES_NUM_BIN_PROFILE = [] 
FEATURES_NUM_ENCODED = [ 'date_processor_bin',   'days_between_bin_decline_type',
                         'days_between_country_decline_type',  
                          'days_between_att_num_decline_type', 
                        'days_between_site_decline_code', 'days_between_bin_decline_code', 'days_between_bin_failed_message',
                        'days_between_site_decline_type'] + FEATURES_NUM_BIN_PROFILE


# FEATURES_GROUPED = [ ['days_between', 'renew_att_num', 'failed_decline_type'], 
#                     ['days_between', 'site_id', 'failed_decline_type'],
#                     ['days_between', 'issuer_country', 'failed_decline_type'],
#                     ['days_between', 'bin', 'failed_decline_type'],
#                     ['days_between', 'bin', 'failed_response_code'],
#                     ['day_of_month', 'bin', 'failed_decline_type'],
#                     ['days_between', 'renew_att_num', 'failed_response_code']
#                     ]
FEATURES_GROUPED = []

#'days_between_site_decline_code', 'days_between_bin_decline_code',
FEATURES_CAT_ENCODED = [ 'is_expired']
FEATURES_ENCODED = FEATURES_CAT_ENCODED + FEATURES_NUM_ENCODED#,  'is_expired'

# FEATURES_NUM_ENCODED = ['payment_mid_bin']

FEATURES = FEATURES_CAT + FEATURES_ENCODED


from sklearn.model_selection import GridSearchCV
# from spark_sklearn import GridSearchCV

from src.web.preprocessing import PreProcessing
from src.web.preprocessing import make_pipeline
from sklearn.preprocessing import Imputer

# FIELDS = FEATURES_CAT + FEATURES_NUM + ['transaction_date_in_string', 'cc_expiration_date', 'failed_attempt_date', 'failed_cc_expiration_date'] + FEATURES_NUM_ENCODED + FEATURES_NUM_CALCULATED
ADDITIONAL_FIELDS = ['site_id', 'renew_att_num', 'failed_decline_type',  'bin', 'payment_service_id', 'failed_day_of_month', 'failed_response_code', 'payment_amount_usd', 'issuer_country',  'failed_response_message','days_between',  'transaction_date_in_string', 'cc_expiration_date', 'failed_attempt_date']
FIELDS = FEATURES_CAT + FEATURES_NUM + ADDITIONAL_FIELDS #+ FEATURES_NUM_BIN_PROFILE

df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

# features_dict = {'FEATURES_CAT': FEATURES_CAT, 'FEATURES_NUM':FEATURES_NUM, 'FEATURES_ENCODED':FEATURES_ENCODED, 'FEATURES_NUM_ENCODED':FEATURES_NUM_ENCODED, 'FEATURES_NUM_CALCULATED':FEATURES_NUM_CALCULATED, 'FEATURES_FLOAT': FEATURES_FLOAT}
# features_dict_key = 'preprocessing__features_dict'

features_dict = {'LABEL': LABEL, 'FIELDS': FIELDS ,'FEATURES_CAT': FEATURES_CAT, 'FEATURES_NUM':FEATURES_NUM, 'FEATURES_ENCODED':FEATURES_ENCODED, 'FEATURES_NUM_ENCODED':FEATURES_NUM_ENCODED, 'FEATURES_NUM_CALCULATED':FEATURES_NUM_CALCULATED, 'FEATURES_FLOAT': FEATURES_FLOAT}
features_dict_key = 'preprocessing__features_dict'
features_dict['df_bin_profile'] = None #bin_profile
features_dict['df_decline_type'] = df_decline_type
features_dict['payment_mid_bin_dict'] = {}#payment_mid_bin_dict
features_dict['FEATURES_NUM_BIN_PROFILE'] = FEATURES_NUM_BIN_PROFILE
features_dict['FEATURES_GROUPED'] = FEATURES_GROUPED

# features_dict['group_dict'] = {"date_bin_dict": date_bin_dict, \
#                                "date_processor_bin_dict": date_processor_bin_dict }

features_dict['group_dict'] = {
                               "date_processor_bin_dict": date_processor_bin_dict, \
                               "days_between_site_decline_type_dict": days_between_site_decline_type_dict, \
                               "days_between_site_decline_code_dict": days_between_site_decline_code_dict, \
                               "days_between_att_num_decline_type_dict": days_between_att_num_decline_type_dict, \
                               "days_between_bin_decline_type_dict": days_between_bin_decline_type_dict, "days_between_country_decline_type_dict":days_between_country_decline_type_dict, \
                               "days_between_bin_failed_message_dict":days_between_bin_failed_message_dict,  \
                               "days_between_bin_decline_code_dict": days_between_bin_decline_code_dict }

#                               "max_per_date_dict": max_per_date_dict , \
# features_dict['group_dict'] = {}

In [41]:
retry_success_2019_07

,received_date,site_id,subscription_id,added_expiry_years,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,funding_source,issuer_country,merchant_number,new_status,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,transaction_date_in_string,transaction_hour,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between
0,2019-07-25,adbehap,10001242501,NaN,Banco Popular y de Desarrollo Comunal,CR,491261.0,Visa,Visa Classic,Consumer,Debit - NON USA Consumer Debit - No Pin Access,0222,NaN,NaN,25,Debit,CR,DR_IRELAND_ROW,Declined,NaN,USD,Visa,adyen,2,2019-07-25 01:01:39,1,0,2019-07-18 00:58:29,18,530.0,27001: Transaction refused,txn_refused,7
1,2019-07-27,adbehbr,10006249801,NaN,Banco Cooperativo Sicredi S.A.,BR,496045.0,Visa,Visa Business,Business,Credit - True credit (No PIN/Signature capability),0626,NaN,NaN,27,Credit,BR,75FA81DE-5AC5-5DCF-E97C-8F80A08ADE1E,Completed,NaN,BRL,Visa,pagador,2,2019-07-27 02:16:11,2,1,2019-07-20 04:16:01,20,530.0,7: Do Not Honor,do_not_honor,7
2,2019-07-18,tmapac,10011699401,NaN,AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED,AU,514045.0,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,Credit - True credit (No PIN/Signature capability),09/20,NaN,NaN,18,Credit,AU,I7A0010,Declined,NaN,AUD,MasterCard,nab,2,2019-07-18 09:00:18,9,0,2019-07-13 02:52:04,13,530.0,27001: Transaction refused,txn_refused,5
3,2019-07-25,tmapac,10011699401,NaN,AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED,AU,514045.0,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,Credit - True credit (No PIN/Signature capability),09/20,NaN,NaN,25,Credit,AU,I7A0010,Declined,NaN,AUD,MasterCard,nab,3,2019-07-25 01:03:02,1,0,2019-07-18 09:00:20,18,530.0,27001: Transaction refused,txn_refused,7
4,2019-07-02,adbehme,10019376401,3,Arab National Bank,SA,455035.0,Visa,Visa Rewards,Consumer,Credit - True credit (No PIN/Signature capability),0421,NaN,NaN,2,Credit,SA,DR_IRELAND_ROW,Declined,10.34,SAR,Visa,adyen,2,2019-07-02 01:09:02,1,0,2019-06-25 02:16:24,25,401.0,"07 : Pickup card, special condition",pickup_card,7
5,2019-07-09,adbehme,10019376401,3,Arab National Bank,SA,455035.0,Visa,Visa Rewards,Consumer,Credit - True credit (No PIN/Signature capability),0421,NaN,NaN,9,Credit,SA,DR_IRELAND_ROW,Declined,10.36,SAR,Visa,adyen,3,2019-07-09 01:10:34,1,0,2019-07-02 01:09:02,2,401.0,"07 : Pickup card, special condition",pickup_card,7
6,2019-07-16,adbehme,10019376401,3,Arab National Bank,SA,455035.0,Visa,Visa Rewards,Consumer,Credit - True credit (No PIN/Signature capability),0421,NaN,NaN,16,Credit,SA,DR_IRELAND_ROW,Declined,10.36,SAR,Visa,adyen,4,2019-07-16 01:42:46,1,0,2019-07-09 01:10:34,9,401.0,"07 : Pickup card, special condition",pickup_card,7
7,2019-07-23,adbehme,10019376401,3,Arab National Bank,SA,455035.0,Visa,Visa Rewards,Consumer,Credit - True credit (No PIN/Signature capability),0421,NaN,NaN,23,Credit,SA,DR_IRELAND_ROW,Declined,NaN,SAR,Visa,adyen,5,2019-07-23 01:33:30,1,0,2019-07-16 01:42:46,16,401.0,Blocked Card: Blocked Card,blocked_first_used,7
8,2019-07-25,adbehme,10019376401,3,Arab National Bank,SA,455035.0,Visa,Visa Rewards,Consumer,Credit - True credit (No PIN/Signature capability),0421,NaN,NaN,25,Credit,SA,DR_IRELAND_ROW,Declined,NaN,SAR,Visa,adyen,6,2019-07-25 01:01:01,1,0,2019-07-23 01:33:30,23,401.0,"07 : Pickup card, special condition",pickup_card,2
9,2019-07-04,adbehap,10025916001,3,CITIBANK SINGAPORE LIMITED,SG,542550.0,MasterCard,World MasterCard Card,Consumer,Credit - True credit (No PIN/Signature capability),0620,NaN,NaN,4,Credit,SG,DR_IRELAND_ROW,Completed,20.36,SGD,MasterCard,adyen,2,2019-07-04 01:12:09,1,1,2019-06-27 01:23:02,27,521.0,51 : Insufficient funds/over credit limit,Insufficient Funds,7


In [ ]:
""" Train the model"""

from catboost import CatBoostClassifier, Pool
import src.web.preprocessing
from src.web.preprocessing import PreProcessing
from src.web.train_util import *
from importlib import import_module
import sys

classifier = CatBoostClassifier

cat_features_len = len(FEATURES_CAT) + len(FEATURES_CAT_ENCODED)
input_data = retry_success_2019  #retry_success[~retry_success["subscription_id"].isin(duplicate_subs)]
scale_pos_weight = (input_data[LABEL].value_counts(normalize=True)[0.0] / input_data[LABEL].value_counts(normalize=True)[1.0] ) + 0.5

features_dict['use_cat_encoder'] = False
_preProcessor = PreProcessing(None).fit(input_data, input_data['success'], features_dict=features_dict)            
_x_eval = _preProcessor.transform(retry_success_08)
_y_eval = retry_success_08["success"]

alg_name = 'catboostclassifier'

tuned_parameters = {}

best_parameters = {
              'learning_rate': 0.1, 
              'depth': 7,
              'iterations': 1000,
              'random_seed': 8,
              'scale_pos_weight': scale_pos_weight,
              'subsample': 0.3,
              'bagging_temperature': 3,
              'rsm': 0.4,
              'eval_metric': 'Recall',
              'l2_leaf_reg': 8.0
              }

#               'eval_metric': 'Recall',
model_file = ''
model_id = 'ML-BR-1'
version = get_latest_version(model_id, MODEL_TYPE) + 1
model_name = model_id + '.' + str(version)

# features_dict['eval_metric'] = 'auc'
# features_dict['early_stopping_rounds'] = None #1600
# features_dict['eval_set'] = None #Pool(_x_eval, _y_eval) 

cat_features = list(range(0,cat_features_len))

fit_params = {
    f"{alg_name}__verbose": True,
    f"{alg_name}__cat_features": cat_features,
    f"{alg_name}__eval_set": Pool(_x_eval, _y_eval, cat_features)
}


features_dict['fit_params'] = fit_params


clf, result_d = build_and_train(input_data, classifier, tuned_parameters, alg_name, model_file, best_param=best_parameters, features_dict=features_dict, test_data=retry_success_09)
model_file, model_file_name = write_model(clf, model_name)
print("model_file is generated: ", model_file)
print("result_dict: ", result_d)


df_bin_profile is NONE
self.features_encoded: ['is_expired']
# Finish handle_feat_encoded.
self.features_all:  None
In fit, self.features_cat_and_encoded: ['merchant_number', 'day_of_month', 'is_expired']
['merchant_number', 'day_of_month', 'is_expired']
fit df[self.features_cat_and_encoded] size: (4484942, 3)
# not using cat encoder
# Finish handle_feat_encoded.
# transform_time: 0.048616886138916016
          merchant_number day_of_month is_expired  date_processor_bin  \
0          dr_ireland_row           29      false           46.926615   
1          dr_ireland_row           30      false           50.877193   
2  1883448942-brl-pacific           30      false           33.489461   
3              dr_ireland           31      false           16.666667   
4  digital_river_software           31      false           46.926615   

   days_between_bin_decline_type  days_between_country_decline_type  \
0                      31.081081                          16.555829   
1             

/var/spark/jupyter-notebooks/src/web/preprocessing.py:485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[feat] = df[feat].fillna('').astype(str).str.replace('.0', '', regex=False)
/var/spark/jupyter-notebooks/src/web/preprocessing.py:503: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[IS_EXPIRED] = df[~df['cc_expiration_date'].isna()].apply(is_expired, axis=1)
/var/spark/jupyter-notebooks/src/web/preprocessing.py:512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

# Finish handle_feat_encoded.


/var/spark/jupyter-notebooks/src/web/preprocessing.py:554: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[BIN] = pd.to_numeric(df[BIN], errors='coerce')
/var/spark/jupyter-notebooks/src/web/preprocessing.py:555: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[BIN] = df[BIN].astype(str).str.replace('.0', '', regex=False)
/var/spark/jupyter-notebooks/src/web/preprocessing.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

self.features_all:  None
In fit, self.features_cat_and_encoded: ['merchant_number', 'day_of_month', 'is_expired']
['merchant_number', 'day_of_month', 'is_expired']
fit df[self.features_cat_and_encoded] size: (4484942, 3)
# not using cat encoder
# Finish handle_feat_encoded.


In [36]:
result_d

{'hyper_params': {'learning_rate': 0.1,
  'depth': 7,
  'iterations': 1000,
  'random_seed': 8,
  'scale_pos_weight': 8.968394996569378,
  'subsample': 0.3,
  'bagging_temperature': 3,
  'rsm': 0.4,
  'eval_metric': 'AUC:hints=skip_train~false',
  'l2_leaf_reg': 9.0},
 'training_time': 3083.814452648163,
 'accuracy_dummy': 0.8668165520238702,
 'train_accuracy': 0.7462747567304103,
 'test_accuracy': 0.7164993689770742,
 'train_class_report': '             precision    recall  f1-score   support\n\n          0       0.97      0.74      0.84   4011267\n          1       0.27      0.82      0.41    473675\n\navg / total       0.90      0.75      0.79   4484942\n',
 'test_class_report': '             precision    recall  f1-score   support\n\n          0       0.96      0.70      0.81    103712\n          1       0.30      0.82      0.44     15935\n\navg / total       0.87      0.72      0.76    119647\n',
 'feature_importance_columns': "Index(['merchant_number', 'is_expired', 'days_between

In [31]:
result_d

{'hyper_params': {'learning_rate': 0.1,
  'depth': 7,
  'iterations': 1000,
  'random_seed': 8,
  'scale_pos_weight': 16.166666666666664,
  'subsample': 0.3,
  'bagging_temperature': 3,
  'rsm': 0.4,
  'eval_metric': 'AUC:hints=skip_train~false',
  'l2_leaf_reg': 9.0},
 'training_time': 48.575135469436646,
 'accuracy_dummy': 0.8668165520238702,
 'train_accuracy': 0.85,
 'test_accuracy': 0.6865613011609151,
 'train_class_report': '             precision    recall  f1-score   support\n\n          0       1.00      0.84      0.91        94\n          1       0.29      1.00      0.44         6\n\navg / total       0.96      0.85      0.89       100\n',
 'test_class_report': '             precision    recall  f1-score   support\n\n          0       0.91      0.70      0.80    103712\n          1       0.23      0.57      0.33     15935\n\navg / total       0.82      0.69      0.73    119647\n',
 'feature_importance_columns': "Index(['date_mid_bin', 'is_expired', 'days_between_site_decline_c

In [27]:
result_d

{'hyper_params': {'learning_rate': 0.1,
  'depth': 7,
  'iterations': 1000,
  'random_seed': 8,
  'scale_pos_weight': 7.131698969672881,
  'subsample': 0.3,
  'bagging_temperature': 3,
  'rsm': 0.4,
  'eval_metric': 'AUC:hints=skip_train~false',
  'l2_leaf_reg': 7.0,
  'custom_metric': ['AUC:hints=skip_train~false']},
 'training_time': 8545.070698738098,
 'accuracy_dummy': 0.8668165520238702,
 'train_accuracy': 0.833775106297692,
 'test_accuracy': 0.7340342841859804,
 'train_class_report': '             precision    recall  f1-score   support\n\n          0       0.99      0.82      0.90   7374940\n          1       0.44      0.95      0.60   1112074\n\navg / total       0.92      0.83      0.86   8487014\n',
 'test_class_report': '             precision    recall  f1-score   support\n\n          0       0.96      0.72      0.83    103712\n          1       0.31      0.80      0.45     15935\n\navg / total       0.87      0.73      0.77    119647\n',
 'feature_importance_columns': "Ind

In [38]:
result_d

{'hyper_params': {'learning_rate': 0.1,
  'depth': 7,
  'iterations': 1000,
  'random_seed': 8,
  'scale_pos_weight': 4.024046435951139,
  'subsample': 0.4,
  'bagging_temperature': 2,
  'rsm': 0.4,
  'custom_metric': ['AUC:hints=skip_train~false']},
 'training_time': 5021.908931016922,
 'accuracy_dummy': 0.8668165520238702,
 'train_accuracy': 0.8308965704202117,
 'test_accuracy': 0.7882270345265656,
 'train_class_report': '             precision    recall  f1-score   support\n\n          0       0.98      0.80      0.88   4011267\n          1       0.57      0.94      0.71   1138256\n\navg / total       0.89      0.83      0.84   5149523\n',
 'test_class_report': '             precision    recall  f1-score   support\n\n          0       0.95      0.80      0.87    103712\n          1       0.35      0.72      0.47     15935\n\navg / total       0.87      0.79      0.81    119647\n',
 'feature_importance_columns': "Index(['date_bin', 'days_between_att_num_decline_type', 'payment_servic

In [30]:
result_d

{'hyper_params': {'learning_rate': 0.18,
  'depth': 9,
  'iterations': 1000,
  'random_seed': 8,
  'scale_pos_weight': 8.5,
  'subsample': 0.4,
  'bagging_temperature': 1,
  'rsm': 0.5,
  'custom_metric': ['AUC:hints=skip_train~false']},
 'training_time': 4488.8786380290985,
 'accuracy_dummy': 0.8668165520238702,
 'train_accuracy': 0.6928904933306592,
 'test_accuracy': 0.6608606985549157,
 'train_class_report': '             precision    recall  f1-score   support\n\n          0       0.97      0.58      0.73   4011267\n          1       0.49      0.95      0.65   1672953\n\navg / total       0.83      0.69      0.70   5684220\n',
 'test_class_report': '             precision    recall  f1-score   support\n\n          0       0.96      0.63      0.76    103712\n          1       0.26      0.84      0.40     15935\n\navg / total       0.87      0.66      0.72    119647\n',
 'feature_importance_columns': "Index(['days_between_att_num_decline_type', 'date_bin', 'renew_att_num',\n       'd

In [ ]:
"""Upload model to Nexus repo and insert the model info into Cassandra table"""
import json

# start_date = '2018-08-01'
start_date = '2019-01-01'

# end_date = '2019-04-30'
end_date = '2019-07-31'

try:
    repo_path = upload_artifact(model_file_name)
    preprocess_repo_path = handle_preprocessing_file(model_id, version)
    size_desc = str(", original size: %s (fail: %s, success: %s), balanced_size: %s" % (original_size, fail_size, success_size, original_size))
    desc = '{}_{}_for_calendar retry model,  eval_metric= auc, with no date_increment, no payment amount and bin profile). {}'.format(start_date, end_date, size_desc)
    hyper_params = result_d.pop('hyper_params', None)
    extended_att = {"preprocess_repo_path": preprocess_repo_path, "input_features": INPUT_FEATURES}
    insert_model_info(model_id, version, repo_path, desc=desc, model_type=MODEL_TYPE,eval_metrics=json.dumps(result_d), 
                      hyper_parameter=json.dumps(hyper_params), extended_att=json.dumps(extended_att), features_dict=features_dict, algorithm='CatBoostClassifier')
    
except Exception as ex:
    print(str(ex))
    if not hyper_params:
        result_d['hyper_params'] = hyper_params